pip install gymnasium[box2d]
pip install stable-baselines3[extra]: The deep reinforcement learning library.
pip install huggingface_sb3:

Usando env.step(action), realizamos esta ação no ambiente e obtemos

- observation: O novo estado (st+1)
- reward:A recompensa que obtemos após executar a ação
- terminated: Indica se o episódio foi encerrado (o agente atingiu o estado terminal)
- truncated:Introduzido com esta nova versão, ele indica um limite de tempo ou se um agente sai dos limites do ambiente, por exemplo.
- info: Um dicionário que fornece informações adicionais (depende do ambiente).

In [12]:
import gymnasium

# from huggingface_sb3 import load_from_hub, package_to_hub
# from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
from tensorboard import notebook
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import StopTrainingOnNoModelImprovement, StopTrainingOnRewardThreshold, EvalCallback, CheckpointCallback
import gymnasium as gym

In [ ]:

log_dir = "./logs/"
# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v3")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

- Coordenada de almofada horizontal (x)
- Coordenada de almofada vertical (y)
- Velocidade horizontal (x)
- Velocidade vertical (y)
- Ângulo
- Velocidade angular
- Se o ponto de contato da perna esquerda tocou o solo (booleano)
- Se o ponto de contato da perna direita tocou o solo (booleano)

In [ ]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v3")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

- Ação 0: Não fazer nada,
- Ação 1: Disparar motor de orientação à esquerda,
- Ação 2: Disparar o motor principal,
- Ação 3: Disparar motor de orientação correta.

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

Ambiente Vetorizado
- Criamos um ambiente vetorizado (um método para empilhar vários ambientes independentes em um único ambiente) de 16 ambientes. Dessa forma, teremos experiências mais diversas durante o treinament

In [16]:
# Create the environment
#env = make_vec_env('LunarLander-v3', n_envs=16)

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

# Callback para salvar checkpoints durante o treinamento
checkpoint_callback = CheckpointCallback(
    save_freq=100,  # Frequência de salvamento em passos de treinamento. Aqui, o modelo será salvo a cada 100 passos.
    save_path="./logs/",  # Caminho onde os checkpoints serão salvos.
    name_prefix="rl_model",  # Prefixo usado nos nomes dos arquivos salvos. Exemplo: rl_model_step_x.zip.
    save_replay_buffer=True,  # Salvar o buffer de replay (usado em algoritmos off-policy como DQN ou SAC).
    save_vecnormalize=True,  # Salvar o estado da normalização de vetores (caso seja usada com `VecNormalize`).
)

# Callback para avaliação do desempenho do modelo durante o treinamento
eval_callback = EvalCallback(
    env,  # Ambiente usado para avaliação.
    eval_freq=10000,  # Frequência de avaliação, em passos de treinamento. Aqui, o modelo será avaliado a cada 10.000 passos.
    callback_on_new_best=callback_on_best,  # Callback opcional a ser chamado quando o modelo atingir uma nova melhor avaliação.
    callback_after_eval=stop_train_callback,  # Callback opcional a ser executado após cada avaliação, como parar o treinamento se uma condição for atingida.
    verbose=1,  # Nível de verbosidade. `1` exibe informações sobre o progresso da avaliação.
    best_model_save_path=os.path.join(model_dir, f"{args.gymenv}_{args.sb3_algo}"),  
    # Caminho para salvar o melhor modelo encontrado durante as avaliações. 
    # Aqui, combina o diretório base (model_dir) com o nome do ambiente (gymenv) e do algoritmo (sb3_algo).
)


In [ ]:


env = gym.make('LunarLander-v3',  render_mode="rgb_array")
# Save a checkpoint every 1000 steps
env = gym.wrappers.RecordVideo(env=env, video_folder="./video_trainig/", name_prefix="test-video", episode_trigger=lambda x: x % 100 == 0)
# Instantiate the agent
env = Monitor(env)

model = PPO('MlpPolicy', env, verbose=1, n_steps=200, tensorboard_log=log_dir)
# Train the agent
model.learn(total_timesteps=int(100_000), callback=model.save("ppo_cartpole"),)


In [18]:
# model.save("ppo_cartpole")

In [ ]:
# model2 = PPO.load("ppo_cartpole")

# env = gym.make('LunarLander-v3')
# obs = env.reset()
# print(obs[0])
# for _ in range(20):
#     action, _states = model2.predict(obs[0])
#     print(action)
#     observation , rewards, terminated ,truncated , info = env.step(action)
#     #env.render("human")

## Análise dos Gráficos de Treinamento

### 1. `rollout/ep_len_mean` (Comprimento Médio do Episódio)
#### **Descrição**
- Este gráfico mostra o comprimento médio dos episódios ao longo do treinamento.
- Um comprimento maior geralmente indica que o agente está melhorando seu desempenho no ambiente.

#### **Indicadores**
- **Bons:**
  - Aumento constante no comprimento médio ao longo do tempo.
  - Indica que o agente está sobrevivendo mais tempo ou completando tarefas de forma eficiente.
- **Ruins:**
  - Estagnação ou queda no comprimento médio.
  - Pode indicar que o agente está tendo dificuldades para aprender.

---

### 2. `rollout/ep_rew_mean` (Recompensa Média por Episódio)
#### **Descrição**
- Este gráfico mede a recompensa média obtida pelo agente em cada episódio.
- Recompensas mais altas indicam que o agente está aprendendo a maximizar o objetivo definido.

#### **Indicadores**
- **Bons:**
  - Recompensas crescentes ao longo do treinamento.
  - Indica que o agente está melhorando suas decisões no ambiente.
- **Ruins:**
  - Recompensas negativas ou que não apresentam crescimento.
  - Sinal de que o agente está estagnado ou não está aprendendo de forma eficaz.

---

### 3. `time/fps` (Frames por Segundo)
#### **Descrição**
- Este gráfico exibe a eficiência computacional do treinamento, medindo o número de frames por segundo (FPS) processados.

#### **Indicadores**
- **Bons:**
  - FPS elevado e estável (acima de 300).
  - Indica que o treinamento está sendo executado de forma eficiente.
- **Ruins:**
  - Quedas abruptas ou FPS consistentemente abaixo de 200.
  - Pode ser um indicativo de sobrecarga no hardware ou aumento da complexidade computacional.

---

### Conclusão Geral
1. **`rollout/ep_len_mean`**: O comprimento médio dos episódios está aumentando, o que é um sinal positivo de aprendizado.
2. **`rollout/ep_rew_mean`**: A recompensa média também está crescendo, indicando que o agente está melhorando seu desempenho.
3. **`time/fps`**: Apesar de uma queda gradual no FPS, ele permanece em um nível aceitável (~285 FPS), mantendo o treinamento eficiente.

Esses gráficos mostram que o treinamento está funcionando bem, tanto em termos de aprendizado do agente quanto de eficiência computacional.


In [ ]:

%load_ext tensorboard
%tensorboard --logdir ./logs/